# Build the database from calendars

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

# set the folder
folder = Path('/run/user/1000/gvfs/smb-share:server=istore,share=microscopy_group_share,user=jeromeb/Admin/Usage Stats/2024')
booking_types = ['default','service', 'maintenance', 'training']
divisions = ['Unknown','Cell Biology','Neurobiology','PNAS','Structural Studies','University']
groups = pd.read_csv(folder/'groups.csv')
users = pd.read_csv(folder/'users.csv')
calendars = pd.read_csv(folder/'calendars.csv')
# get a cookie from the curl command copied from firefox dev tools
with open('cookie.txt','r') as f:
    curl_str = f.read()
# curl_str = "curl 'https://mercury.mrc-lmb.cam.ac.uk/ical' --compressed -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8' -H 'Accept-Language: en-GB,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Connection: keep-alive' -H 'Referer: https://mercury.mrc-lmb.cam.ac.uk/integration/linux.html?lang=en' -H 'Cookie: SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1' -H 'Priority: u=0, i'"
# cookie = 'SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14'
db_file = 'bookings.db'

## Update or initialize the database.

- Load all the calendars into the database.

In [52]:
from usagestats.bookingdb import BookingDB

with BookingDB(db_file, curl_str=curl_str) as bookings:
    bookings.create_calendars_from_df(calendars)
    bookings.create_instruments()
    bookings.create_booking_types(booking_types)
    bookings.create_division(divisions)
    bookings.create_groups(groups)
    df, duplicates = bookings.create_events(users)


spinning_disk
zeiss_sim
nikon_isim
leica_sted
zeiss_780_uv
zeiss_axioplan
zeiss_axio_imager
zeiss_710_upright
analysis_pc4
andor_bc43
nikon_w1
analysis_pc3
zeiss_2_photon
zeiss_900_airyscan_ai
analysis_pc2
olympus_slide_scanner
olympus_slide_scanner
zeiss_sim_analysis_pc
Zeiss_780
opm
Zeiss_710
analysis_pc1
leica_sp8
nikon_ti2
nikon_hca
nanoimager
nikon_tirf
nikon_n-storm
zeiss_880_airyscan
error in events UNIQUE constraint failed: events.guid 1bd6da22-7442-4877-99e4-b67e14101ee3
error in events UNIQUE constraint failed: events.guid 51A021B1-3F5B-434A-9E92-27FC3AC85433
error in events UNIQUE constraint failed: events.guid E28C7718-05D9-42DB-B55A-77C964A36964
error in events UNIQUE constraint failed: events.guid 9B4E5561-DD07-402F-BB3F-413CC0ED5B81
error in events UNIQUE constraint failed: events.guid 83C7346E-5241-4DE9-AFBA-C9737AD45474
error in events UNIQUE constraint failed: events.guid 4745F794-D617-49A0-9770-1A4ECB008D3B
error in events UNIQUE constraint failed: events.guid 22E40E

Analysis of duplicates at all time

In [69]:
pd.concat(df[df['guid']==x].iloc[0].to_frame().T for x in duplicates).groupby('user')['hours'].agg(['count','sum'])

,count,sum
user,,
Ana Crespillo,13,62.5
Ben Sutcliffe,188,2647.5
Gerard Crossan,1,3.0
Nathan Richoz,1,1.0
Nathaniel Hoyle,1,73.0


## Update user information

In [70]:
with BookingDB(db_file, curl_str=curl_str) as bookings:
    bookings.update_users_group_from_df(users)

In [ ]:
with BookingDB(db_file, cookie) as bookings:
    print(bookings.cookie)
    

In [ ]:
with BookingDB(db_file) as bookings:
    bookings.cursor.execute("""SELECT rowid FROM divisions WHERE name=='Cell Biology'""")
    r = bookings.cursor.fetchone()
    print(bookings.get_division_id("Cell Biology"))


In [ ]:
bookings.cur.execute("""SELECT rowid,name FROM booking_types WHERE name=='toto'""")
rows = bookings.cur.fetchall()
#rows[0][0]
#for row in rows:
#    print(row)
len(rows)

In [ ]:
try :
    bookings.cur.execute('''INSERT INTO booking_types (name) VALUES (?)''', ('service',))
except :
    pass



In [ ]:
bookings.create_booking_types(booking_types)


In [ ]:
del bookings